Imports

In [245]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split

#preprossors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer

#models
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR


from sklearn.model_selection import RandomizedSearchCV

#Other
from os import path, getcwd
import sql_functions

ModuleNotFoundError: No module named 'sql_functions'

Open file

In [219]:
# path is ../sql/db/__NAMEOFCSV__.csv
pathToParent = path.dirname(getcwd())
csvFile = pathToParent + "/sqlite/db/Fintech.csv"
completeDF = pd.read_csv(csvFile, names=['name','date', 'daybefore', 'dayafter', 'monthafter', 'threemonthsafter', 'transcript' ])



Create Test and trianing sets

In [208]:
trainDF, testDF = train_test_split(completeDF, test_size=.2, random_state=435)

print(trainDF)
trainDF = trainDF.dropna()
testDF = testDF.dropna()

trainX = trainDF.drop(columns=['name','dayafter', 'monthafter', 'threemonthsafter'])

long_y_train = trainDF['threemonthsafter']
medium_y_train = trainDF['monthafter']
short_y_train = trainDF['dayafter']

testX = testDF.drop(columns=['name','dayafter', 'monthafter', 'threemonthsafter'])

long_y_test = testDF['threemonthsafter']
medium_y_test = testDF['monthafter']
short_y_test = testDF['dayafter']


         name        date  daybefore  dayafter   monthafter  threemonthsafter  \
22      Apple  1525132800    162.130    175.23   184.973636        187.764844   
96      Intel  1651104000     45.480     44.99    43.597182         40.736687   
49  Microsoft  1580256000    163.780    174.05   179.093636        166.017656   
37      Apple  1643241600    163.500    165.71   169.241136        167.397109   
30      Apple  1588204800    284.730    286.25   308.689545        343.352086   
..        ...         ...        ...       ...          ...               ...   
84      Intel  1556150400     58.520     52.74    48.141364         47.541406   
12     Google  1612224000   1844.585   2065.61  2062.099800       2127.138712   
89      Intel  1595462400     61.310     52.15    49.715909         50.560133   
69        AMD  1595894400     69.280     75.50    81.110686         81.644009   
66        AMD  1572307200     33.160     32.93    37.167273         42.420000   

                           

Create Dummy models

In [209]:
shortDummy = DummyClassifier()
mediumDummy = DummyClassifier()
longDummy = DummyClassifier()

shortDummy.fit(trainX, short_y_train)
mediumDummy.fit(trainX, medium_y_train)
longDummy.fit(trainX, long_y_train)

print(shortDummy.predict(trainX))
print(mediumDummy.predict(testX))
print(longDummy.predict(testX))

[21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49
 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49
 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49
 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49
 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49
 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49
 21.49 21.49 21.49 21.49 21.49 21.49 21.49 21.49]
[23.41909091 23.41909091 23.41909091 23.41909091 23.41909091 23.41909091
 23.41909091 23.41909091 23.41909091 23.41909091 23.41909091 23.41909091
 23.41909091 23.41909091 23.41909091 23.41909091 23.41909091 23.41909091
 23.41909091 23.41909091]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Create preprocessing

In [210]:
trainX.transcript = trainX['transcript'].values.astype('U')
testX.transcript = testX['transcript'].values.astype('U')


In [211]:
preprocessor = ColumnTransformer([('transcript', CountVectorizer(stop_words="english", strip_accents='ascii'),'transcript'),
                                ('daybefore', 'passthrough', ['daybefore']),
                                ('date', "drop", 'date')])
countvec = CountVectorizer(stop_words='english', strip_accents='ascii')
X_counts = countvec.fit_transform(trainX['transcript'])


Create pipelines

In [216]:
pipe = make_pipeline(
    preprocessor, 
    LinearRegression())

pipe.fit(trainX, short_y_train)

predictions = pipe.predict(testX)

for i in range(len(testX['daybefore'].array)):
    print(testX['daybefore'].array[i], predictions[i], short_y_test.array[i])


375.0 379.8797295403162 411.535
24.18 23.511833388127208 17.92
261.66 257.6807095839688 256.078
27.9 24.864648850165132 28.95
1115.0 1146.4553957000803 1048.33
74.23 97.03745413447157 78.47
58.27 56.977568362298996 56.2
55.25 30.691547037732438 50.39
52.21 15.986681217243834 48.76
51.86 41.77616837643939 55.44
85.655 90.28306706575125 98.18
213.85 205.56947055738672 207.67
48.41 34.9270369417153 46.85
34.14 17.03228641698895 32.08
61.04 79.67131277340478 66.57
55.18 36.47591641830203 54.58
986.27 987.1126955771502 1030.99
143.6 146.6633308641827 139.52
150.96 136.72143827416141 148.2
310.06 322.3008750906864 324.45


Creating a grid search to find best hyperperameters. 

In [213]:
param_dist = {"columntransformer__transcript__max_features" : range(100, 10000, 100)}

random_search = RandomizedSearchCV(
    pipe, param_dist, cv=4, n_jobs=-1, scoring= "roc_auc")

shortModel = random_search.fit(trainX, short_y_train)



C:\Users\svenw\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [214]:
print(shortModel.predict(testX))

print(short_y_test.array)


[ 378.31409238   24.17460525  255.4673893    24.28704901 1147.30346159
   99.9969797    58.2289537    29.21548686   11.01782264   41.21354189
   90.32932949  203.47275857   34.60336664   14.36689401   82.75975052
   33.90365147  990.36247026  146.61637703  135.19300452  325.08944937]
<PandasArray>
[411.535,   17.92, 256.078,   28.95, 1048.33,   78.47,    56.2,   50.39,
   48.76,   55.44,   98.18,  207.67,   46.85,   32.08,   66.57,   54.58,
 1030.99,  139.52,   148.2,  324.45]
Length: 20, dtype: float64


In [185]:
param_dist = {"columntransformer__transcript__max_features" : range(100, 10000, 100)}

random_search = RandomizedSearchCV(
    pipe, param_dist, cv=4, n_jobs=-1, scoring= "roc_auc")

mediumModel = random_search.fit(trainX, medium_y_train)

C:\Users\svenw\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [186]:
print(mediumModel.predict(testX))

print(medium_y_test.array)

[ 375.19297894   28.35913142  253.35239003   28.04912042 1123.55999398
   78.43807271   49.50894699   83.37174468   52.94486953   51.14149766
   89.26252927  204.25215502   10.40373164   16.74173228   39.33463146
   51.4947357   937.94276333  129.30052601  119.11955365  332.41722584]
<PandasArray>
[ 459.2465681818181, 19.712727272727275, 248.84054545454543,
  27.25909090909091,  1062.898181818182,  81.01350000000001,
  59.50045454545455,  50.26505909090909, 48.388181818181806,
  53.78409090909091,  94.75977272727272,  212.8215909090909,
  49.24727272727273, 31.009318181818177,  66.32863636363638,
  53.64429090909091, 1040.4272727272726, 133.64136363636365,
 147.00363636363633, 315.97636363636354]
Length: 20, dtype: float64


In [187]:
param_dist = {"columntransformer__transcript__max_features" : range(100, 10000, 100)}

random_search = RandomizedSearchCV(
    pipe, param_dist, cv=4, n_jobs=-1, scoring= "roc_auc")

longModel = random_search.fit(trainX, long_y_train)

C:\Users\svenw\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [188]:
print(longModel.predict(testX))

print(long_y_test.array)

[ 3.93578227e+02  3.50826568e+01  2.10247137e+02  3.40189073e+01
  1.11030072e+03  3.38238376e+01  7.15488628e+01  7.61360282e+01
  5.49323495e+01  6.77064688e+01  6.78397151e+01  1.89398504e+02
 -9.80465956e+00 -4.65767388e-01  3.17763208e+01  2.21138001e+01
  9.27408520e+02  1.49883947e+02  9.07189061e+01  3.51454358e+02]
<PandasArray>
[234.55319531250004,  19.63421875000001,  260.1805859374999,
 30.051484375000005,          1060.8325,                0.0,
  60.16468749999998,  51.20227499999999,  47.19499999999999,
  53.24281250000001,       89.126171875,  215.9439062500001,
  52.80156250000002, 30.533984374999992,  58.67562500000001,
       53.787384375, 1065.9328124999995,       128.47078125,
 140.03862500000002,  282.3162499999999]
Length: 20, dtype: float64


Test different models

In [175]:
pipe = make_pipeline(
    preprocessor,
    SVR())
pipe.fit(trainX, short_y_train)
print(pipe.predict(testX))
print(short_y_test.array)

[134.20115974 141.97467077 137.84293319 134.39529757 182.44430798
 182.13104907 140.20352379 142.55686067 139.3722675  146.13975821
 139.20891787 139.77374891 136.86688055 136.52717382 182.49779804
 147.32367126 171.18364818 134.85348607 139.64776856 136.52717382
 145.97943728 145.50468164 171.35512124 135.71528418 134.72032143
 150.5870323  181.45081703 134.85255594 137.49347046 142.6577931
 142.55686067 139.80094399 145.60991746 135.10172026 134.73133714
 183.02300183 137.44599236 135.44558709 137.44599236 135.70949538]
<PandasArray>
[   54.19,   209.88,    103.8,    52.74,   1271.0,  1667.44,   165.71,
   207.67,   148.03,   288.99,   139.39,    148.2,   129.89,   121.63,
  1497.05,   324.45,   3025.0,    58.85,  147.215,   121.63,  307.985,
    282.1, 2726.239,    78.73,    55.44,  411.535,  1048.33,  62.9999,
   109.26,   209.55,   207.67,   161.84,   286.25,    78.47,     75.5,
   1276.0,    96.43,    21.49,    96.43,    98.18]
Length: 40, dtype: float64


Create predictions

Add predictions to database

In [241]:
completeDF.sort_values('date', ascending=False)
companyCount = completeDF['name'].nunique()

predictionDict = {}
rowIndex = 1
while len(predictionDict) < companyCount:
    row = completeDF.iloc[rowIndex].to_frame().T
    if row['name'].iloc[0] not in predictionDict:
        predictions = [shortModel.predict(row)[0], mediumModel.predict(row)[0], longModel.predict(row)[0]]
        predictionDict[row['name'].iloc[0]] = predictions
    
    rowIndex += 1



In [242]:
predictionDict

{'Google': [1127.4200000000008, 1092.9122727272716, 1076.083941015856],
 'Apple': [159.10000000000065, 168.30809523809535, 170.81992582841744],
 'Microsoft': [64.35499999999772, 64.18886363636393, 65.01846684511169],
 'AMD': [24.174605252008583, 28.35913141900148, 35.082656806257795],
 'Intel': [41.213541894606244, 51.14149765944388, 67.70646881775117]}